# #13 Extensions

1. [Extension](#Extension)
2. [Cython](#Cython)
3. [Pypy](#Pypy)
4. [Foreign Function Interfaces](#Foreign-Function-Interfaces)
	1. [ctypes](#ctypes)
	2. [cffi](#cffi)

### Extension

![c](https://pics.me.me/assembly-pimpmygun-doctorn-c-python-19950587.png)

Homework example

In [ ]:
#include <Python.h>
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include "deviceapps.pb-c.h"

#define MAGIC  0xFFFFFFFF
#define DEVICE_APPS_TYPE 1

typedef struct pbheader_s {
    uint32_t magic;
    uint16_t type;
    uint16_t length;
} pbheader_t;
#define PBHEADER_INIT {MAGIC, 0, 0}


// https://github.com/protobuf-c/protobuf-c/wiki/Examples
void example() {
    DeviceApps msg = DEVICE_APPS__INIT;
    DeviceApps__Device device = DEVICE_APPS__DEVICE__INIT;
    void *buf;
    unsigned len;

    char *device_id = "e7e1a50c0ec2747ca56cd9e1558c0d7c";
    char *device_type = "idfa";
    device.has_id = 1;
    device.id.data = (uint8_t*)device_id;
    device.id.len = strlen(device_id);
    device.has_type = 1;
    device.type.data = (uint8_t*)device_type;
    device.type.len = strlen(device_type);
    msg.device = &device;

    msg.has_lat = 1;
    msg.lat = 67.7835424444;
    msg.has_lon = 1;
    msg.lon = -22.8044005471;

    msg.n_apps = 3;
    msg.apps = malloc(sizeof(uint32_t) * msg.n_apps);
    msg.apps[0] = 42;
    msg.apps[1] = 43;
    msg.apps[2] = 44;
    len = device_apps__get_packed_size(&msg);

    buf = malloc(len);
    device_apps__pack(&msg, buf);

    fprintf(stderr,"Writing %d serialized bytes\n",len); // See the length of message
    fwrite(buf, len, 1, stdout); // Write to stdout to allow direct command line piping

    free(msg.apps);
    free(buf);
}

// Read iterator of Python dicts
// Pack them to DeviceApps protobuf and write to file with appropriate header
// Return number of written bytes as Python integer
static PyObject* py_deviceapps_xwrite_pb(PyObject* self, PyObject* args) {
    const char* path;
    PyObject* o;

    if (!PyArg_ParseTuple(args, "Os", &o, &path))
        return NULL;

    printf("Write to: %s\n", path);
    Py_RETURN_NONE;
}

// Unpack only messages with type == DEVICE_APPS_TYPE
// Return iterator of Python dicts
static PyObject* py_deviceapps_xread_pb(PyObject* self, PyObject* args) {
    const char* path;

    if (!PyArg_ParseTuple(args, "s", &path))
        return NULL;

    printf("Read from: %s\n", path);
    Py_RETURN_NONE;
}


static PyMethodDef PBMethods[] = {
     {"deviceapps_xwrite_pb", py_deviceapps_xwrite_pb, METH_VARARGS, "Write serialized protobuf to file from iterator"},
     {"deviceapps_xread_pb", py_deviceapps_xread_pb, METH_VARARGS, "Deserialize protobuf from file, return iterator"},
     {NULL, NULL, 0, NULL}
};


PyMODINIT_FUNC initpb(void) {
     (void) Py_InitModule("pb", PBMethods);
}


In [ ]:
from setuptools import setup, Extension

module1 = Extension("pb",
                    sources=["pb.c", "deviceapps.pb-c.c"],
                    extra_compile_args=["-g"],
                    libraries=["protobuf-c"],
                    library_dirs=["/usr/lib"],
                    include_dirs=["/usr/include/google/protobuf-c/"],
                    )

setup(name="pb",
      version="1.0",
      description="Protobuf (de)serializer",
      test_suite="tests",
      ext_modules=[module1])

In [ ]:
import os
import unittest

import pb
MAGIC = 0xFFFFFFFF
DEVICE_APPS_TYPE = 1
TEST_FILE = "test.pb.gz"


class TestPB(unittest.TestCase):
    deviceapps = [
        {"device": {"type": "idfa", "id": "e7e1a50c0ec2747ca56cd9e1558c0d7c"},
         "lat": 67.7835424444, "lon": -22.8044005471, "apps": [1, 2, 3, 4]},
        {"device": {"type": "gaid", "id": "e7e1a50c0ec2747ca56cd9e1558c0d7d"}, "lat": 42, "lon": -42, "apps": [1, 2]},
        {"device": {"type": "gaid", "id": "e7e1a50c0ec2747ca56cd9e1558c0d7d"}, "lat": 42, "lon": -42, "apps": []},
        {"device": {"type": "gaid", "id": "e7e1a50c0ec2747ca56cd9e1558c0d7d"}, "apps": [1]},
    ]

    def tearDown(self):
        os.remove(TEST_FILE)

    def test_write(self):
        bytes_written = pb.deviceapps_xwrite_pb(self.deviceapps, TEST_FILE)
        self.assertTrue(bytes_written > 0)
        # check magic, type, etc.

    @unittest.skip("Optional problem")
    def test_read(self):
        pb.deviceapps_xwrite_pb(self.deviceapps, TEST_FILE)
        for i, d in pb.deviceapps_xread_pb(TEST_FILE):
            self.assertEqual(d, self.deviceapps[i])


C++ binding

In [ ]:
#include <Python.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>

#include <boost/scoped_ptr.hpp>
#include <boost/property_tree/ptree.hpp>
#include <boost/property_tree/json_parser.hpp>
#include <boost/foreach.hpp>

#include <s2polygon.h>
#include <s2polygonbuilder.h>
#include <s2regioncoverer.h>

PyDoc_STRVAR(py_cover_geojson_doc,
    "cover_geojson(path, only_id=True, max_cells=4096, min_level=0, max_level=30, level_mod=1)\n"
    "--\n\n"
    "Computes Google S2 covering for polygons defined in GeoJSON specified by path.\n"
    "Return list of results.\n"
    "only_id - return id or full cell representation\n"
    "max_cells - upper limit for number of cells to cover a region\n"
    "min_level - minumum cell level to cover\n"
    "max_level - maximum cell level to cover\n"
    "level_mod - if specified, then only cells where (level - min_level) is a multiple of "
    "`level_mod` will be used (default 1). This effectively allows the branching "
    "factor of the S2CellId hierarchy to be increased. Currently the only "
    "parameter values allowed are 1, 2, or 3, corresponding to branching factors "
    "of 4, 16, and 64 respectively.\n"
);

static PyObject* py_cover_geojson(PyObject* self, PyObject* args, PyObject *kwargs) {
    const char* path;
    int only_id = 1;
    int max_cells = 4096;
    int min_level = 0;
    int max_level = 30;
    int level_mod = 1;

    static char *kwlist[] = {"path", "only_id", "max_cells", "min_level", "max_level", "level_mod", NULL};
    if (!PyArg_ParseTupleAndKeywords(args, kwargs, "s|BIIII", kwlist, &path, &only_id, &max_cells, &min_level, &max_level, &level_mod)) {
        return NULL;
    }

    if (only_id == 0) {
        PyErr_SetString(PyExc_NotImplementedError, "Can only return cell ids");
        return NULL;      
    }

    if (access(path, F_OK) == -1 ) {
        PyErr_Format(PyExc_OSError, "No such file: %s", path);
        return NULL;
    }

    S2RegionCoverer coverer;
    coverer.set_level_mod(level_mod);
    coverer.set_min_level(min_level);
    coverer.set_max_level(max_level);
    coverer.set_max_cells(max_cells);

    using boost::property_tree::ptree;
    std::ifstream jsonFile(path);
    ptree pt;
    try {
        read_json(jsonFile, pt);
    } catch(boost::property_tree::json_parser::json_parser_error &je) {
        PyErr_SetString(PyExc_ValueError, "Cannot parse JSON from file");
        return NULL;          
    }
    double coordinate[2];
    std::vector<S2Point> s2points_vector;
    PyObject *res = PyList_New(0);
    BOOST_FOREACH(const ptree::value_type &polygonsList, pt.get_child("geojson.geometries..coordinates")) {
        unsigned i = 0;
        BOOST_FOREACH(const ptree::value_type &ring, polygonsList.second) {
            // exterior ring
            if (i == 0) {
                // missed code...
                int r;
                unsigned j;
                for (j = 0; j < cellids_vector.size(); j++) {
                    if (only_id) {
                        r = PyList_Append(res, PyLong_FromUnsignedLongLong(cellids_vector[j].id()));
                        if (r == -1) {
                            PyErr_SetString(PyExc_RuntimeError, "Cannot append to list");
                            return NULL;
                        }
                    }
                }
                s2points_vector.clear();
            }
            i++;
        }
    }
    return res;
}


static PyMethodDef S2Methods[] = {
    {"cover_geojson", (PyCFunction)py_cover_geojson, METH_VARARGS | METH_KEYWORDS, py_cover_geojson_doc},
    {NULL, NULL, 0, NULL}
};


PyMODINIT_FUNC initrbs2(void) {
    (void) Py_InitModule3("rbs2", S2Methods, "Google S2 for RB");
}


In [ ]:
import sys
import subprocess
from setuptools import setup, Extension

if "clean" not in sys.argv:
    subprocess.call(["make", "-C", "s2-geometry-library/geometry/"])

module1 = Extension("rbs2",
                    sources=["rbs2.cpp"],
                    extra_compile_args=[
                        "-O2",
                        "-fPIC",
                        "-DHAVE_CONFIG_H",
                        "-Wno-deprecated",
                        "-Wno-write-strings",
                        "-fno-strict-aliasing",
                        "-pthread"
                    ],
                    libraries=[
                        "s2",
                        "s2cellid",
                        "google-util-coding",
                        "google-util-math",
                        "google-strings",
                        "google-base",
                        "ssl",
                        "z",
                    ],
                    library_dirs=["s2-geometry-library/geometry/"],
                    include_dirs=[".", "s2-geometry-library/geometry/"],
                    )

setup(name="rbs2",
      version="0.1",
      description="Google S2 binding for RB",
      test_suite="tests",
      ext_modules=[module1])


In [ ]:
import os
import unittest

import rbs2

TEST_GEOJSON = "tests/geo.json"


class TestS2(unittest.TestCase):

    def test_cover(self):
        expected = [9727172662748250112L, 9727178160306388992L, 9727180359329644544L, 9727207847120338944L,
                    9727229837352894464L, 9727251827585449984L, 9727278215864516608L, 9727287011957538816L,
                    9727313400236605440L, 9727335390469160960L]
        cell_ids = rbs2.cover_geojson(TEST_GEOJSON, only_id=True, max_cells=10)
        self.assertEqual(sorted(cell_ids), expected)
        self.assertEqual(len(rbs2.cover_geojson(TEST_GEOJSON)), 2908)

    def test_nofile(self):
        self.assertRaises(OSError, rbs2.cover_geojson, "nosuchfile")

    def test_badjson(self):
        f = "bad.json"
        open(f, "w").close()
        self.assertRaises(ValueError, rbs2.cover_geojson, f)
        os.remove(f)

    def test_not_ids(self):
        self.assertRaises(NotImplementedError, rbs2.cover_geojson, TEST_GEOJSON, False)


Build dict

In [ ]:
#include <Python.h>
#include <stdio.h>
#include <stdlib.h>
#include <inttypes.h>
#include <string.h>
#include <zlib.h>
#include "rbfile.h"
#define FREQUENCY_COUNT 33 

typedef struct __attribute__((packed)){
    uint64_t id;
    unsigned short shows;
    unsigned short clicks;
} record;

typedef struct {
    unsigned long long all_shows, all_clicks, uniq_shows, uniq_clicks;
    unsigned long long shows_frequency[FREQUENCY_COUNT], clicks_frequency[FREQUENCY_COUNT];
} counters;
    
static void count(rbfile_t *fp, counters *cnt) {
// ...
}

static PyObject * py_get_counters(PyObject *self, PyObject *args, PyObject *kwargs) {
    static PyObject *error;
    char *in, *etype;
    uint32_t eid;
    uint32_t as_json = 0;
    uint32_t key_size = 8;
    static char *kwlist[] = {"filepath", "entity_type", "entity_id", "key_size", "as_json", NULL};
    if (!PyArg_ParseTupleAndKeywords(args, kwargs, "ssI|IB", kwlist, &in, &etype, &eid, &key_size, &as_json)) {
        PyErr_SetString(error, "Can't parse args");
        return NULL;
    }
    counters cnt;
    cnt.all_shows = cnt.all_clicks = cnt.uniq_shows = cnt.uniq_clicks = 0;
    memset(cnt.shows_frequency, 0, sizeof(cnt.shows_frequency));
    memset(cnt.clicks_frequency, 0, sizeof(cnt.clicks_frequency));

    rbfile_t *fp = rbfile_new();
    fp->open(fp, in, "rb");
    if (fp != 0) {
        count(fp, &cnt);
    }
    else {
        PyErr_SetString(error, "Can't open file");
        return NULL;        
    }
    fp->close(fp);
    rbfile_delete(&fp);
    
    if (!as_json) {
        PyObject *slist = PyList_New(FREQUENCY_COUNT);
        PyObject *clist = PyList_New(FREQUENCY_COUNT);
        if (!slist || !clist) {
            PyErr_SetString(error, "Can't alloc memory");
            return NULL;
        }

        uint32_t f;
        for (f = 0; f < FREQUENCY_COUNT; ++f) {
            if (PyList_SetItem(slist, f, Py_BuildValue("K", cnt.shows_frequency[f])) == -1 
                || PyList_SetItem(clist, f, Py_BuildValue("K", cnt.clicks_frequency[f])) == -1) {
                PyErr_SetString(error, "Can't add result to response");
                Py_DECREF(slist);
                Py_DECREF(clist);
                return NULL;
            }
        }

        PyObject * result = Py_BuildValue("{s:s,s:I,s:K,s:K,s:K,s:K,s:O,s:O}", 
                             "type", etype, 
                             "id", eid, 
                             "shows_all", cnt.all_shows, 
                             "clicks_all", cnt.all_clicks, 
                             "shows_uniq", cnt.uniq_shows, 
                             "clicks_uniq", cnt.uniq_clicks, 
                             "shows_freq", slist, 
                             "clicks_freq", clist
                            );
        Py_DECREF(slist);
        Py_DECREF(clist);
        return result;
    }
    else {
        char json[1024];
        char *pos = json;
        pos += sprintf(pos, "{\"type\": \"%s\", \"id\": %d, \"shows_uniq\": %llu, \"shows_all\": %llu, \"clicks_uniq\": %llu, \"clicks_all\": %llu", etype, eid, cnt.uniq_shows, cnt.all_shows, cnt.uniq_clicks, cnt.all_clicks);

        pos += sprintf(pos, ", \"shows_freq\": [%llu", cnt.shows_frequency[0]);
        uint32_t f;
        for (f = 1; f < FREQUENCY_COUNT; ++f)
            pos += sprintf(pos, ", %llu", cnt.shows_frequency[f]);
        pos += sprintf(pos, "]");

        pos += sprintf(pos, ", \"clicks_freq\": [%llu", cnt.clicks_frequency[0]);
        for (f = 1; f < FREQUENCY_COUNT; ++f)
            pos += sprintf(pos, ", %llu", cnt.clicks_frequency[f]);
        pos += sprintf(pos, "]}");
        return Py_BuildValue("s", json);
    }
    Py_RETURN_NONE;
}

static PyMethodDef Methods[] = {
    {"get_counters", (PyCFunction)py_get_counters, METH_VARARGS | METH_KEYWORDS, "Parse rbuniq file and count useful info."},
    {NULL, NULL, 0, NULL}
};

PyMODINIT_FUNC initrbuniq(void) {
    PyObject *m;
    
    m = Py_InitModule("rbuniq", Methods);
    if (!m) {
        return;
    }
}

int main(int argc, char **argv) {
    Py_SetProgramName(argv[0]);
    Py_Initialize();
    initrbuniq();
    return 0;
}


In [ ]:
from distutils.core import setup, Extension


module1 = Extension('rbuniq',
                    extra_compile_args=['-DHAVE_ZLIB=1', '-DHAVE_LZ4=1'],
                    include_dirs=['../../src/rbfile/'],
                    sources=['rbuniq.c', '../../src/rbfile/rbfile.c'],
                    extra_link_args=['-lz', '-llz4'])
setup(name='rbuniq', description='python wrapper for rbuniq', ext_modules=[module1])


Custom error and GIL release

In [ ]:
static PyObject *error;

static PyObject *py_targets_negative(PyObject *self, PyObject *args) {

	char *all, *in, *out;
	int has_header = 1;
	int key64 = 0;
	char errorstr[8192];
	int has_error = 0;
	errorstr[0] = 0;

	if (!PyArg_ParseTuple(args, "sss|BB", &all, &in, &out, &has_header, &key64)) {
		PyErr_SetString(error, "Can't parse args");
		return NULL;
	}

	Py_BEGIN_ALLOW_THREADS

	int key_size = sizeof(uint32_t);
	if (key64)
		key_size = sizeof(uint64_t);

	if (targets_negative(all, in, out, has_header, key_size, errorstr, sizeof(errorstr)) != 0) {
		has_error = 1;
	}

	Py_END_ALLOW_THREADS

	if (has_error) {
		PyErr_SetString(error, errorstr);
		return NULL;
	}

	Py_INCREF(Py_None);
	return Py_None;
}

PyMODINIT_FUNC initrbcapacityd_py(void) {

	PyObject *m;

	m = Py_InitModule("rbcapacityd_py", RbMethods);
	if (!m)
		return;

	error = PyErr_NewException("rbcapacityd_py.error", NULL, NULL);
	Py_INCREF(error);
	PyModule_AddObject(m, "error", error);
}


int main(int argc, char **argv) {

	Py_SetProgramName(argv[0]);
	Py_Initialize();
	initrbcapacityd_py();
	return 0;
}

Add constants

In [ ]:
PyMODINIT_FUNC initrbgraph_c(void) {
  PyObject *m;

  m = Py_InitModule("rbgraph_c", RbMethods);
  if (!m) {
    return;
  }
  
  RbError = PyErr_NewException("rbgraph_c.error", NULL, NULL);
  Py_INCREF(RbError);
  PyModule_AddObject(m, "error", RbError);

  PyModule_AddIntConstant (m, "STAT_SHOW", STAT_SHOW);
  PyModule_AddIntConstant (m, "STAT_CLK", STAT_CLK);
  PyModule_AddIntConstant (m, "STAT_EMPTY", STAT_EMPTY);
  PyModule_AddIntConstant (m, "STAT_BANNER_EVENT", STAT_BANNER_EVENT);

}


int main(int argc, char **argv) {
  Py_SetProgramName(argv[0]);
  Py_Initialize();
  initrbgraph_c();
  return 0;
}

Iterator

In [ ]:
#include <Python.h>

#include <stdlib.h>
#include <stdint.h>
#include <inttypes.h>
#include <stdbool.h>
#include <stdio.h>
#include <string.h>
#include <ctype.h>

#include <tarantool/tnt.h>
#include <tarantool/tnt_net.h>
#include <tarantool/tnt_snapshot.h>

static PyObject *SnapshotError;

typedef struct {
    PyObject_HEAD
    struct tnt_stream stream;
    struct tnt_iter iter;
    struct tnt_iter iter_tuple;
    int open_exception;
    int iter_created;
} SnapshotIterator;

static PyMethodDef SnapshotIterator_Methods[] = {
    {NULL}  /* Sentinel */
};

static int SnapshotIterator_init(SnapshotIterator *self, PyObject *args);
PyObject* SnapshotIterator_del(SnapshotIterator *self);
PyObject* SnapshotIterator_iter(SnapshotIterator *self);
PyObject* SnapshotIterator_iternext(SnapshotIterator *self);

static PyTypeObject SnapshotIterator_Type = {
    PyObject_HEAD_INIT(NULL)
    0,                         /*ob_size*/
    "tarantool_snapshot.SnapshotIterator",      /*tp_name*/
    sizeof(SnapshotIterator),      /*tp_basicsize*/
    0,                         /*tp_itemsize*/
    (destructor)SnapshotIterator_del,             /*tp_dealloc*/
    0,                         /*tp_print*/
    0,                         /*tp_getattr*/
    0,                         /*tp_setattr*/
    0,                         /*tp_compare*/
    0,                         /*tp_repr*/
    0,                         /*tp_as_number*/
    0,                         /*tp_as_sequence*/
    0,                         /*tp_as_mapping*/
    0,                         /*tp_hash */
    0,                         /*tp_call*/
    0,                         /*tp_str*/
    0,                         /*tp_getattro*/
    0,                         /*tp_setattro*/
    0,                         /*tp_as_buffer*/
    Py_TPFLAGS_DEFAULT | Py_TPFLAGS_HAVE_ITER,
        /*
         * tp_flags: Py_TPFLAGS_HAVE_ITER tells python to
         * use tp_iter and tp_iternext fields.
        */
    "SnapshotIterator",                /* tp_doc */
    0,                         /* tp_traverse */
    0,                         /* tp_clear */
    0,                         /* tp_richcompare */
    0,                         /* tp_weaklistoffset */
    (getiterfunc)SnapshotIterator_iter,  /* tp_iter: __iter__() method */
    (iternextfunc)SnapshotIterator_iternext,  /* tp_iternext: next() method */
    SnapshotIterator_Methods,          /* tp_methods */
    0,                         /* tp_members */
    0,                         /* tp_getset */
    0,                         /* tp_base */
    0,                         /* tp_dict */
    0,                         /* tp_descr_get */
    0,                         /* tp_descr_set */
    0,                         /* tp_dictoffset */
    (initproc)SnapshotIterator_init,   /* tp_init */
    0                          /* tp_alloc */
};

static int SnapshotIterator_init(SnapshotIterator *self, PyObject *args) {
    char *filename = NULL;
    //size_t len = 0;
    self->open_exception = 0;
    self->iter_created = 0;
    
    if (!PyArg_ParseTuple(args, "s", &filename)) {
        return 1;
    }
    
    tnt_snapshot(&(self->stream));
    
    if (tnt_snapshot_open(&(self->stream), filename) == -1) {
        self->open_exception = 1;
        return 1;
    }

    return 0;
}

void create_iterator_if_required(SnapshotIterator* self) {
    if (self->iter_created) {
        return;
    }
    tnt_iter_storage(&(self->iter), &(self->stream));
    self->iter_created = 1;
}

PyObject* SnapshotIterator_iter(SnapshotIterator* self) {
    if (self->open_exception) {
        PyErr_Format(SnapshotError, "Can't open snapshot");
        return NULL;
    }
    create_iterator_if_required(self);
    Py_INCREF(self);
    return (PyObject*)self;
}

PyObject* SnapshotIterator_iternext(SnapshotIterator* self) {
    create_iterator_if_required(self);
    if (tnt_next(&(self->iter))) {
        struct tnt_iter_storage *is = TNT_ISTORAGE(&(self->iter));
        struct tnt_stream_snapshot* ss = TNT_SSNAPSHOT_CAST(TNT_ISTORAGE_STREAM(&(self->iter)));
        
        PyObject *tuple = PyList_New(0);
        PyObject *s;
        
        tnt_iter(&(self->iter_tuple), &is->t);
        while (tnt_next(&(self->iter_tuple))) {
            char *data = TNT_IFIELD_DATA(&(self->iter_tuple));
            uint32_t size = TNT_IFIELD_SIZE(&(self->iter_tuple));
            s = PyString_FromStringAndSize(data, size);
            PyList_Append(tuple, s);
            Py_DECREF(s);
            //printf("size: %d\n", size);
        }
        if (self->iter_tuple.status == TNT_ITER_FAIL){
            PyErr_Format(SnapshotError, "parsing error");
            return NULL;
        }
        tnt_iter_free(&(self->iter_tuple));
        
        PyObject* tuple_as_tuple = PyList_AsTuple(tuple);
        PyObject* ret = Py_BuildValue("(I,O)", ss->log.current.row_snap.space, tuple_as_tuple);
        Py_DECREF(tuple_as_tuple);
        Py_DECREF(tuple);
        return ret;
    }
    if (self->iter.status == TNT_ITER_FAIL) {
        PyErr_Format(SnapshotError, "Parsing failed: %s", tnt_snapshot_strerror(&(self->stream)));
        return NULL;
    }
    return NULL;
}

PyObject* SnapshotIterator_del(SnapshotIterator* self) {
    if (self->iter_created) {
        tnt_iter_free(&(self->iter));
    }
    tnt_stream_free(&(self->stream));
    PyObject_Del(self);
    
    Py_RETURN_NONE;
}

static PyMethodDef TarantoolSnapshot_Module_Methods[] = {
    {NULL}  /* Sentinel */
};

PyMODINIT_FUNC inittarantool_snapshot(void) {
    PyObject *m;

    SnapshotIterator_Type.tp_new = PyType_GenericNew;
    if (PyType_Ready(&SnapshotIterator_Type) < 0)  return;

    m = Py_InitModule("tarantool_snapshot", TarantoolSnapshot_Module_Methods);
    if (!m) {
        return;
    }

    Py_INCREF(&SnapshotIterator_Type);
    PyModule_AddObject(m, "iter", (PyObject *)&SnapshotIterator_Type);

    SnapshotError = PyErr_NewException("tarantool_snapshot.SnapshotError", NULL, NULL);
    Py_INCREF(SnapshotError);
    PyModule_AddObject(m, "SnapshotError", SnapshotError); 
}

int main(int argc, char **argv) {
    Py_SetProgramName(argv[0]);
    Py_Initialize();
    inittarantool_snapshot();
    return 0;
}


New types

In [ ]:
#include <Python.h>
#include "structmember.h"

typedef struct {
    PyObject_HEAD
    PyObject *first; /* first name */
    PyObject *last;  /* last name */
    int number;
} Noddy;

static void
Noddy_dealloc(Noddy* self)
{
    Py_XDECREF(self->first);
    Py_XDECREF(self->last);
    Py_TYPE(self)->tp_free((PyObject*)self);
}

static PyObject *
Noddy_new(PyTypeObject *type, PyObject *args, PyObject *kwds)
{
    Noddy *self;

    self = (Noddy *)type->tp_alloc(type, 0);
    if (self != NULL) {
        self->first = PyString_FromString("");
        if (self->first == NULL) {
            Py_DECREF(self);
            return NULL;
        }

        self->last = PyString_FromString("");
        if (self->last == NULL) {
            Py_DECREF(self);
            return NULL;
        }

        self->number = 0;
    }

    return (PyObject *)self;
}

static int
Noddy_init(Noddy *self, PyObject *args, PyObject *kwds)
{
    PyObject *first=NULL, *last=NULL, *tmp;

    static char *kwlist[] = {"first", "last", "number", NULL};

    if (! PyArg_ParseTupleAndKeywords(args, kwds, "|OOi", kwlist,
                                      &first, &last,
                                      &self->number))
        return -1;

    if (first) {
        tmp = self->first;
        Py_INCREF(first);
        self->first = first;
        Py_XDECREF(tmp);
    }

    if (last) {
        tmp = self->last;
        Py_INCREF(last);
        self->last = last;
        Py_XDECREF(tmp);
    }

    return 0;
}


static PyMemberDef Noddy_members[] = {
    {"first", T_OBJECT_EX, offsetof(Noddy, first), 0,
     "first name"},
    {"last", T_OBJECT_EX, offsetof(Noddy, last), 0,
     "last name"},
    {"number", T_INT, offsetof(Noddy, number), 0,
     "noddy number"},
    {NULL}  /* Sentinel */
};

static PyObject *
Noddy_name(Noddy* self)
{
    static PyObject *format = NULL;
    PyObject *args, *result;

    if (format == NULL) {
        format = PyString_FromString("%s %s");
        if (format == NULL)
            return NULL;
    }

    if (self->first == NULL) {
        PyErr_SetString(PyExc_AttributeError, "first");
        return NULL;
    }

    if (self->last == NULL) {
        PyErr_SetString(PyExc_AttributeError, "last");
        return NULL;
    }

    args = Py_BuildValue("OO", self->first, self->last);
    if (args == NULL)
        return NULL;

    result = PyString_Format(format, args);
    Py_DECREF(args);

    return result;
}

static PyMethodDef Noddy_methods[] = {
    {"name", (PyCFunction)Noddy_name, METH_NOARGS,
     "Return the name, combining the first and last name"
    },
    {NULL}  /* Sentinel */
};

static PyTypeObject NoddyType = {
    PyVarObject_HEAD_INIT(NULL, 0)
    "noddy.Noddy",             /* tp_name */
    sizeof(Noddy),             /* tp_basicsize */
    0,                         /* tp_itemsize */
    (destructor)Noddy_dealloc, /* tp_dealloc */
    0,                         /* tp_print */
    0,                         /* tp_getattr */
    0,                         /* tp_setattr */
    0,                         /* tp_compare */
    0,                         /* tp_repr */
    0,                         /* tp_as_number */
    0,                         /* tp_as_sequence */
    0,                         /* tp_as_mapping */
    0,                         /* tp_hash */
    0,                         /* tp_call */
    0,                         /* tp_str */
    0,                         /* tp_getattro */
    0,                         /* tp_setattro */
    0,                         /* tp_as_buffer */
    Py_TPFLAGS_DEFAULT |
        Py_TPFLAGS_BASETYPE,   /* tp_flags */
    "Noddy objects",           /* tp_doc */
    0,                         /* tp_traverse */
    0,                         /* tp_clear */
    0,                         /* tp_richcompare */
    0,                         /* tp_weaklistoffset */
    0,                         /* tp_iter */
    0,                         /* tp_iternext */
    Noddy_methods,             /* tp_methods */
    Noddy_members,             /* tp_members */
    0,                         /* tp_getset */
    0,                         /* tp_base */
    0,                         /* tp_dict */
    0,                         /* tp_descr_get */
    0,                         /* tp_descr_set */
    0,                         /* tp_dictoffset */
    (initproc)Noddy_init,      /* tp_init */
    0,                         /* tp_alloc */
    Noddy_new,                 /* tp_new */
};

static PyMethodDef module_methods[] = {
    {NULL}  /* Sentinel */
};

#ifndef PyMODINIT_FUNC	/* declarations for DLL import/export */
#define PyMODINIT_FUNC void
#endif
PyMODINIT_FUNC
initnoddy2(void)
{
    PyObject* m;

    if (PyType_Ready(&NoddyType) < 0)
        return;

    m = Py_InitModule3("noddy2", module_methods,
                       "Example module that creates an extension type.");

    if (m == NULL)
        return;

    Py_INCREF(&NoddyType);
    PyModule_AddObject(m, "Noddy", (PyObject *)&NoddyType);
}

### References

* https://www.crumpington.com/blog/2014/10-19-high-performance-python-extensions-part-1.html
* http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html
* https://www.youtube.com/watch?v=phe1s6p38gk
* https://www.paypal-engineering.com/2016/09/22/python-by-the-c-side/
* http://pythonextensionpatterns.readthedocs.io/en/latest/index.html
* https://docs.python.org/2/extending/newtypes.html
* https://llllllllll.github.io/c-extension-tutorial/index.html
* https://www.python.org/dev/peps/pep-0007/
* https://github.com/MSeifert04/iteration_utilities/blob/master/src/_module.c

### Summary

* to write CPython extension you only need to know C and Python
* lots of boilerplate code =(
* C API is actually quite simple and powerful
* But you need to be careful, especially with memory management

### Cython

![cytohon](http://www.programering.com/images/remote/ZnJvbT1jbmJsb2dzJnVybD1jbWJ3NVNPMk1UTXpVak0yRWpOeVFUTTJFekwyQUROeEFqTXZNVE8xZ3pNMDh5WnZ4bVl2MDJiajV5WnZ4bVkwbG1iajV5Y2xkV1l0bDJMdm9EYzBSSGE.jpg)

__Warning__: Not all python code will see massive performance improvements when compiled with Cython.They are only useful for CPU bound, not for I/O bound or network bound.

https://ru.wikipedia.org/wiki/%D0%9C%D0%BD%D0%BE%D0%B6%D0%B5%D1%81%D1%82%D0%B2%D0%BE_%D0%96%D1%8E%D0%BB%D0%B8%D0%B0

![julia](https://upload.wikimedia.org/wikipedia/commons/d/d5/JSr07885.gif)

In [ ]:
# %load julia1.py
"""Julia set generator without optional PIL-based image drawing"""
import time
import calculate

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193


def calc_pure_python(desired_width, max_iterations):
    """Create a list of complex co-ordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (float(x2 - x1) / float(desired_width))
    y_step = (float(y1 - y2) / float(desired_width))
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of co-ordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print "Length of x:", len(x)
    print "Total elements:", len(zs)
    start_time = time.time()
    output = calculate.calculate_z(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print "Took", secs, "seconds"

    validation_sum = sum(output)
    print "Total sum of elements (for validation):", validation_sum


# Calculate the Julia set using a pure Python solution with
# reasonable defaults for a laptop
calc_pure_python(desired_width=1000, max_iterations=300)


In [ ]:
# %load cythonfn.pyx
def calculate_z(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while n < maxiter and abs(z) < 2:
            z = z * z + c
            n += 1
        output[i] = n
    return output


In [ ]:
# %load setup.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext

setup(
    cmdclass={'build_ext': build_ext},
    ext_modules=[Extension(
        "calculate",
        sources=["cythonfn.pyx"],
        extra_compile_args=["-w"])
    ],
)


In [10]:
! python setup.py build_ext --inplace

running build_ext
cythoning cythonfn.pyx to cythonfn.c
building 'calculate' extension
creating build
creating build/temp.macosx-10.12-intel-2.7
cc -fno-strict-aliasing -fno-common -dynamic -arch x86_64 -arch i386 -g -Os -pipe -fno-common -fno-strict-aliasing -fwrapv -DENABLE_DTRACE -DMACOSX -DNDEBUG -Wall -Wstrict-prototypes -Wshorten-64-to-32 -DNDEBUG -g -fwrapv -Os -Wall -Wstrict-prototypes -DENABLE_DTRACE -arch x86_64 -arch i386 -pipe -I/System/Library/Frameworks/Python.framework/Versions/2.7/include/python2.7 -c cythonfn.c -o build/temp.macosx-10.12-intel-2.7/cythonfn.o -w
cc -bundle -undefined dynamic_lookup -arch x86_64 -arch i386 -Wl,-F. build/temp.macosx-10.12-intel-2.7/cythonfn.o -o /Users/s.stupnikov/Dropbox/Teaching/Otus/lection14/calculate.so


In [11]:
! ls -la calculate.so

-rwxr-xr-x@ 1 s.stupnikov  staff  56940  1 окт 16:18 calculate.so


In [12]:
! time python julia1.py

Length of x: 1000
Total elements: 1000000
Took 5.87525296211 seconds
Total sum of elements (for validation): 33219980

real	0m6.993s
user	0m6.575s
sys	0m0.188s


In [13]:
! time python ../lection13/julia.py

Length of x:  1000
Total elements:  1000000
calculate_z_serial_purepython took 10.5714240074 sec

real	0m11.866s
user	0m11.168s
sys	0m0.292s


To generate html annotation we use the command

In [16]:
! cython -a cythonfn.pyx

Each line can be expanded with a double-click to show the generated C code. More yellow means “more calls into the Python virtual machine,” while more white means “more non-Python C code.” The goal is to remove as many of the yellow lines as possible and to end up with as much white as possible

Add some type annotations

In [ ]:
# %load cythonfn2.pyx
def calculate_z(int maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    cdef unsigned int i, n
    cdef double complex z, c
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while n < maxiter and abs(z) < 2:
            z = z * z + c
            n += 1
        output[i] = n
    return output


In [18]:
! time python julia1.py

Length of x: 1000
Total elements: 1000000
Took 2.91271018982 seconds
Total sum of elements (for validation): 33219980

real	0m4.014s
user	0m3.732s
sys	0m0.146s


Rewrite `abs` function

In [ ]:
# %load cythonfn3.pyx
def calculate_z(int maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    cdef unsigned int i, n
    cdef double complex z, c
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while n < maxiter and (z.real * z.real + z.imag * z.imag) < 4:
            z = z * z + c
            n += 1
        output[i] = n
    return output


In [25]:
! time python julia1.py

Length of x: 1000
Total elements: 1000000
Took 0.252919912338 seconds
Total sum of elements (for validation): 33219980

real	0m1.236s
user	0m1.137s
sys	0m0.081s


Integrate with NumPy using `memoryview` annotations

In [ ]:
# %load julia_np.py
"""Julia set generator without optional PIL-based image drawing"""
import time
import numpy as np
import calculate

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193


def calc_pure_python(draw_output, desired_width, max_iterations):
    """Create a list of complex co-ordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (float(x2 - x1) / float(desired_width))
    y_step = (float(y1 - y2) / float(desired_width))
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of co-ordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    zs_np = np.array(zs, np.complex128)
    cs_np = np.array(cs, np.complex128)

    print "Length of x:", len(x)
    print "Total elements:", len(zs)
    start_time = time.time()
    output = calculate.calculate_z(max_iterations, zs_np, cs_np)
    end_time = time.time()
    secs = end_time - start_time
    print "Took", secs, "seconds"

    validation_sum = sum(output)
    print "Total sum of elements (for validation):", validation_sum


# Calculate the Julia set using a pure Python solution with
# reasonable defaults for a laptop
# set draw_output to True to use PIL to draw an image
calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)


In [ ]:
# %load cythonfn_np.pyx
import numpy as np
cimport numpy as np

def calculate_z(int maxiter, double complex[:] zs, double complex[:] cs):
    """Calculate output list using Julia update rule"""
    cdef unsigned int i, n
    cdef double complex z, c
    cdef int[:] output = np.empty(len(zs), dtype=np.int32)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while n < maxiter and (z.real * z.real + z.imag * z.imag) < 4:
            z = z * z + c
            n += 1
        output[i] = n
    return output


In [ ]:
# %load setup.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy as np

setup(
    cmdclass={'build_ext': build_ext},
    ext_modules=[Extension(
        "calculate",
        sources=["cythonfn_np.pyx"],
        extra_compile_args=["-w"],
        include_dirs=[np.get_include()])
    ],
)


In [30]:
! time python julia_np.py

Length of x: 1000
Total elements: 1000000
Took 0.233881950378 seconds
Total sum of elements (for validation): 33219980

real	0m2.154s
user	0m1.771s
sys	0m0.228s


Add OpenMP

In [ ]:
# %load cythonfn_mp.pyx
from cython.parallel import prange
import numpy as np
cimport numpy as np

def calculate_z(int maxiter, double complex[:] zs, double complex[:] cs):
    """Calculate output list using Julia update rule"""
    cdef unsigned int i, length
    cdef double complex z, c
    cdef int[:] output = np.empty(len(zs), dtype=np.int32)
    length = len(zs)
    with nogil:
        for i in prange(length):
            z = zs[i]
            c = cs[i]
            output[i] = 0
            while output[i] < maxiter and (z.real * z.real + z.imag * z.imag) < 4:
                z = z * z + c
                output[i] += 1
    return output


In [ ]:
# %load setup.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy as np

setup(
    cmdclass={'build_ext': build_ext},
    ext_modules=[Extension(
        "calculate",
        sources=["cythonfn_np.pyx"],
        extra_compile_args=["-w", "-fopenmp"],
        extra_link_args=["-fopenmp"],
        include_dirs=[np.get_include()])
    ],
)


### References

* http://cython.readthedocs.io/en/latest/src/tutorial/pure.html
* http://cython.readthedocs.io/en/latest/src/reference/compilation.html
* http://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
* https://www.youtube.com/watch?v=mv0kfiepn3s
* https://habrahabr.ru/company/mailru/blog/242533/
* http://shop.oreilly.com/product/0636920033431.do
* https://www.youtube.com/watch?v=_1MSX7V28Po

### Summary

* Cython supports several methods of compiling to C, some easier than the full-type-annotation method
* Familiarize yourself with the Pure Python Mode if you’d like an easier start to using Cython and 
* Look at pyximport

### Pypy

![pypy2](https://uploads.toptal.io/blog/image/129/toptal-blog-B.png)

![pypy1](http://www.aosabook.org/images/pypy/translation_steps.png)

![pypy3](https://image.slidesharecdn.com/pypy-compilation-framework-121128003538-phpapp01/95/pypy-dynamic-language-compilation-framework-17-638.jpg?cb=1354063042)

### References

* http://www.aosabook.org/en/pypy.html
* https://www.youtube.com/watch?v=VN2s5rr1tnQ
* http://pypy.readthedocs.io/en/latest/cpython_differences.html
* https://www.youtube.com/watch?v=a4M5nsO6ofk
* https://www.youtube.com/watch?v=P5ad6NpjR3M

### Summary

* If you need other packages, anything that is pure Python will probably install, while anything that relies on C extension libraries probably won’t work in a useful way
* PyPy can use a lot of RAM
* PyPy is bound by the global interpreter lock, but the development team are working on a project called software transactional memory (STM) to attempt to remove the need for the GIL
* PyPy does not support projects like numpy in a usable way

### Foreign Function Interfaces

In [ ]:
# %load diffusion.c
void evolve(int Nx, int Ny, double in[][Ny], double out[][Ny], double D, double dt) {
    int i, j;
    double laplacian;
    for (i=1; i<Nx-1; i++) {
        for (j=1; j<Ny-1; j++) {
            laplacian = in[i+1][j] + in[i-1][j] + in[i][j+1] + in[i][j-1] - 4 * in[i][j];
            out[i][j] = in[i][j] + D * dt * laplacian;
        }
    }
}


In [ ]:
# %load Makefile
diffusion.so: diffusion.c
	gcc -O3 -std=gnu99 -c diffusion.c
	gcc -shared -o diffusion.so diffusion.o
	rm -rf diffusion.o


#### ctypes

In [39]:
# %load diffusion_ctypes.py
import numpy as np
import time
import ctypes

grid_shape = (512, 512)
_diffusion = ctypes.CDLL("diffusion.so")

# Create references to the C types that we will need to simplify future code
TYPE_INT = ctypes.c_int
TYPE_DOUBLE = ctypes.c_double
TYPE_DOUBLE_SS = ctypes.POINTER(ctypes.POINTER(ctypes.c_double))

# Initialize the signature of the evolve function to:
# void evolve(int, int, double**, double**, double, double)
_diffusion.evolve.argtypes = [
    TYPE_INT,
    TYPE_INT,
    TYPE_DOUBLE_SS,
    TYPE_DOUBLE_SS,
    TYPE_DOUBLE,
    TYPE_DOUBLE,
]
_diffusion.evolve.restype = None


def evolve(grid, out, dt, D=1.0):
    # First we convert the python types into the relevant C types
    cX = TYPE_INT(grid_shape[0])
    cY = TYPE_INT(grid_shape[1])
    cdt = TYPE_DOUBLE(dt)
    cD = TYPE_DOUBLE(D)
    pointer_grid = grid.ctypes.data_as(TYPE_DOUBLE_SS)
    pointer_out = out.ctypes.data_as(TYPE_DOUBLE_SS)

    # Now we can call the function
    _diffusion.evolve(cX, cY, pointer_grid, pointer_out, cD, cdt)


def run_experiment(num_iterations):
    scratch = np.zeros(grid_shape, dtype=ctypes.c_double)
    grid = np.zeros(grid_shape, dtype=ctypes.c_double)

    block_low = int(grid_shape[0] * .4)
    block_high = int(grid_shape[0] * .5)
    grid[block_low:block_high, block_low:block_high] = 0.005

    start = time.time()
    for i in range(num_iterations):
        evolve(grid, scratch, 0.1)
        grid, scratch = scratch, grid
    return time.time() - start

if __name__ == "__main__":
    t = run_experiment(500)
    print t


0.336709022522


Unless the object you are turning into a ctype object implements a buffer (as do the array module, numpy arrays, cStringIO, etc.), your data will be copied into the new object

`ctypes` struct example

In [ ]:
from ctypes import Structure

class cPoint(Structure):
    _fields_ = ("x", c_int), ("y", c_int)

#### cffi

In [ ]:
# %load diffusion_cffi.py
#!/usr/bin/env python2.7

import numpy as np
import time
from cffi import FFI, verifier

grid_shape = (512, 512)

ffi = FFI()
ffi.cdef(
    r''' void evolve(int Nx, int Ny, double **in, double **out, double D, double dt); ''')
lib = ffi.dlopen("diffusion.so")


def evolve(grid, dt, out, D=1.0):
    X, Y = grid_shape
    pointer_grid = ffi.cast('double**', grid.ctypes.data)
    pointer_out = ffi.cast('double**', out.ctypes.data)
    lib.evolve(X, Y, pointer_grid, pointer_out, D, dt)


def run_experiment(num_iterations):
    scratch = np.zeros(grid_shape, dtype=np.double)
    grid = np.zeros(grid_shape, dtype=np.double)

    block_low = int(grid_shape[0] * .4)
    block_high = int(grid_shape[0] * .5)
    grid[block_low:block_high, block_low:block_high] = 0.005

    start = time.time()
    for i in range(num_iterations):
        evolve(grid, 0.1, scratch)
        grid, scratch = scratch, grid
    return time.time() - start

if __name__ == "__main__":
    t = run_experiment(500)
    print t

    verifier.cleanup_tmpdir()


Inlining

In [43]:
import numpy as np
import time
from cffi import FFI, verifier

grid_shape = (512, 512)

ffi = FFI()
ffi.cdef(
    r'''void evolve(int Nx, int Ny, double **in, double **out, double D, double dt);''')
lib = ffi.verify(r'''
void evolve(int Nx, int Ny, double in[][Ny], double out[][Ny], double D, double dt) {
    int i, j;
    double laplacian;
    for (i=1; i<Nx-1; i++) {
        for (j=1; j<Ny-1; j++) {
            laplacian = in[i+1][j] + in[i-1][j] + in[i][j+1] + in[i][j-1] - 4 * in[i][j];
            out[i][j] = in[i][j] + D * dt * laplacian;
        }
    }
}
''', extra_compile_args=["-O3", ])


def evolve(grid, dt, out, D=1.0):
    X, Y = grid_shape
    pointer_grid = ffi.cast('double**', grid.ctypes.data)
    pointer_out = ffi.cast('double**', out.ctypes.data)
    lib.evolve(X, Y, pointer_grid, pointer_out, D, dt)


def run_experiment(num_iterations):
    scratch = np.zeros(grid_shape, dtype=np.double)
    grid = np.zeros(grid_shape, dtype=np.double)

    block_low = int(grid_shape[0] * .4)
    block_high = int(grid_shape[0] * .5)
    grid[block_low:block_high, block_low:block_high] = 0.005

    start = time.time()
    for i in range(num_iterations):
        evolve(grid, 0.1, scratch)
        grid, scratch = scratch, grid
    return time.time() - start

if __name__ == "__main__":
    t = run_experiment(500)
    print t

    verifier.cleanup_tmpdir()

0.344474077225


### References

* https://www.youtube.com/watch?v=P6qvhoQGaSk
* https://2016.za.pycon.org/talks/20/
* http://eli.thegreenplace.net/2013/03/04/flexible-runtime-interface-to-shared-libraries-with-libffi/
* http://eli.thegreenplace.net/2013/03/09/python-ffi-with-ctypes-and-cffi/
* https://github.com/wolever/python-cffi-example
* https://blog.sentry.io/2016/10/19/fixing-python-performance-with-rust.html
* https://dustymabe.com/2016/09/13/sharing-a-go-library-to-python-using-cffi/
* https://nullprogram.com/blog/2018/05/27/

### Summary

* ctypes is built-in and well suited for small interfaces
* cffi is better ctypes and well suited for interfacing with large C libs
* cffi easily integrated with PyPy
* cffi also useful for integrations with other languages